In [10]:
# Import the library
import cv2

Let's create our EyeTracker class with the track method. First, define the __init__ method and instantiate the faceCascade and Eyecascade using **CascadeClassifier** in opencv and passing the xml file for two cascade. Then, define the track method which takes the image as input and outputs the tracked rectangle points list. In the **track** method,, first detect the face using **facecascade** and detectMultiScale function and returns the rect of face and then pass the ROI to eyecascade to detect the eyes in the faces using **eyecacade** and **detectMultiScale** which returns the rect of the eyes in the face which have been detected 

In [11]:
class EyeTracker:
    def __init__(self, faceCascadePath, eyeCascadePath):
        self.faceCascade = cv2.CascadeClassifier(faceCascadePath)
        self.eyeCascade = cv2.CascadeClassifier(eyeCascadePath)
        
    def track(self, image):
        rects = []
        faceRects = self.faceCascade.detectMultiScale(image, 
                                                     scaleFactor=1.1,
                                                     minNeighbors=5,
                                                     minSize=(30, 30),
                                                     flags=cv2.CASCADE_SCALE_IMAGE)
        for (fX, fY, fW, fH) in faceRects:
            faceROI = image[fY:fY + fH, fX:fX + fW] 
            rects.append((fX, fY, fX+fW, fH+fY))
            
            eyeRects = self.eyeCascade.detectMultiScale(faceROI, 
                                                    scaleFactor=1.1,
                                                    minNeighbors=10,
                                                    minSize=(20, 20),
                                                    flags=cv2.CASCADE_SCALE_IMAGE)    
        
            for (eX, eY, eW, eH) in eyeRects:
                rects.append((fX+eX, fY+eY, fX+eX+eW, fY+eY+eH))
            
        return rects    

### Parameters used in the detectMultiScale function 
**scaleFactor**: How much the image size is reduced at each image scale. This value is used to create the scale pyramid in order to detect faces at multiple scales in the image (some faces may be closer to the foreground, and thus be larger; other faces may be smaller and in the background, thus the usage of varying scales). A value of 1.05 indicates that Jeremy is reducing the size of the image by 5% at each level in the pyramid.                                                                                      

**minNeighbors**: How many neighbors each windo w should have for the area in the window to be considered a face. The cascade classifier will detect multiple windows around a face. This parameter controls how many rectangles (neighbors) need to be detected for the window to be labeled a face.

**minSize**: A tuple of width and height (in pixels) indicating the minimum size of the window. Bounding boxes smaller than this size are ignored. It is a good idea to start with (30, 30) and fine-tune from there.

In [23]:
# Function to resize the video for our convenience
def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    (h, w) = image.shape[:2]
    dim = None
    
    if width is None and height is None:
        return image
    
    if width is None:
        r = height / float(h)
        dim = (int(r * w), height)
    else:
        r = width / float(w)
        dim = (width, int(r * h))
        
    resized = cv2.resize(image, dim, interpolation=inter)
    return resized

In [24]:
# Create the object our our EyeTracker class and arguments as two xml files (face, eye)
et = EyeTracker("cascade/haarcascade_frontalface_default.xml",
                "cascade/haarcascade_eye.xml")

In [25]:
# Choose the video method (Here webcam)
video = cv2.VideoCapture(0)

In [26]:
# Loop to iter the frames
while True:
    (grabbed, frame) = video.read()
    
    # Only work if grabbed is True
    if not grabbed:
        break
    
    # Resize the video   
    frame = resize(frame, width=1000)    
    # Change to gray scale frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # rects - has the rectangle points for face and eyes
    rects = et.track(gray)
    
    # Loop the rects 
    for rect in rects:
        cv2.rectangle(frame, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
    
    # Dispaly the frame
    cv2.imshow("Eye Tracking", frame)    
    
    # Close the webcam if, pressed "q"
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
# Stop the video
video.release()
# Destroy all the windows ehich has been opened by OpenCV
cv2.destroyAllWindows()